[Reference](https://medium.com/spikelab/anomalies-detection-using-river-398544d3536)

In [2]:
!pip install river

     |████████████████████████████████| 2.5 MB 4.9 MB/s 


In [4]:
from river import feature_extraction as fx
from river import compose, stats

features_pipeline = compose.TransformerUnion(
    compose.Select('TotalPaid'),
    compose.Select('PercentageDiscount'),
    compose.Select('StoreId'),
    #A lot of features goes here...
    #
    fx.Agg(on='TotalPaid', by='StoreID', how=RollingMean(window_size=WINDOWS_SIZE)),
    fx.Agg(on='PercentageDiscount', by='StoreID', how=RollingMean(window_size=WINDOWS_SIZE))
    #A lot of features goes here...
    #
)

In [5]:
def build_model(n_tress=25, height=10, window_size=250):
    
  features_pipeline = compose.TransformerUnion(
      compose.Select('TotalPaid'),
      compose.Select('PercentageDiscount'),
      compose.Select('StoreId'),
      #A lot of features goes here...
      #
      fx.Agg(on='TotalPaid', by='StoreID', how=RollingMean(window_size=window_size)),
      fx.Agg(on='PercentageDiscount', by='StoreID', how=RollingMean(window_size=window_size))
      #A lot of features goes here...
      #

    categorical_features = compose.Pipeline(
        compose.SelectType(str),
        preprocessing.OneHotEncoder()
    )

    numerical_features = compose.Pipeline(
            compose.SelectType(numbers.Number),
            preprocessing.MinMaxScaler()
        )

    model = compose.Pipeline(features_pipeline,
                            numerical_features + categorical_features,
                            anomaly.HalfSpaceTrees(n_trees=n_tress, height=height, window_size=window_size))
    return model

In [6]:
def train_model_callback(message, model):
    data = message.data.decode("utf-8")
    transaction = Transaction.parse_raw(data)
    if transaction.TransactionType == "Sale":
        features = create_features(transaction)
        x = features.__dict__ #River need a Dict for training
        score = model.score_one(x)
        #...
        #Generate metrics, insert the prediction into BigQuery
        #and a few steps more.
        #....
        model = model.learn_one(x=x) #Feed River with the need data
        return model